# Task 11.5: FP4 Deep Dive (Blackwell Exclusive!)

**Module:** 11 - Model Quantization & Optimization  
**Time:** 3 hours  
**Difficulty:** ⭐⭐⭐⭐⭐

---

## 🎯 Learning Objectives

By the end of this notebook, you will:
- [ ] Understand NVIDIA FP4 (NVFP4) format and its advantages
- [ ] Learn about MXFP4 (Open Compute Project standard)
- [ ] Apply FP4 quantization using TensorRT Model Optimizer
- [ ] Achieve 3.5× memory reduction with <1% accuracy loss
- [ ] Leverage DGX Spark's unique Blackwell FP4 tensor cores

---

## 📚 Prerequisites

- Completed: Tasks 11.1-11.4
- Knowledge of: Previous quantization methods
- Hardware: **DGX Spark required** (Blackwell GPU for FP4 tensor cores)

### ⚠️ Container Requirements

Ensure you're running in an NGC container with the required flags:

```bash
docker run --gpus all -it --rm \
    -v $HOME/workspace:/workspace \
    -v $HOME/.cache/huggingface:/root/.cache/huggingface \
    --ipc=host \
    nvcr.io/nvidia/pytorch:25.11-py3 \
    jupyter lab --ip=0.0.0.0 --allow-root --no-browser
```

**Important:** The `--ipc=host` flag is required for DataLoader with multiple workers!

---

## 🌍 Real-World Context

### ⭐ This is Your DGX Spark Superpower!

**The Blackwell Advantage:**
- FP4 tensor cores are **exclusive to Blackwell architecture**
- DGX Spark (GB10 Superchip) has **192 5th-gen tensor cores**
- These tensor cores natively support FP4 computation
- **1 PFLOP of FP4 performance** in a desktop form factor!

| Feature | Previous GPUs | Blackwell (DGX Spark) |
|---------|---------------|----------------------|
| FP4 Support | ❌ Software emulation | ✅ Hardware native |
| Memory Reduction | ~2× (INT8) | ~3.5× (FP4) |
| Quality Loss | Higher | <1% with proper calibration |
| Speed | Baseline | ~3× faster prefill |

---

## 🧒 ELI5: What is FP4?

> **Imagine you're a music producer mixing a song...**
>
> **Standard approach (FP16):** Record everything in high quality
> - Crystal clear audio, large files
> - Perfect for the studio master
>
> **INT4 approach:** Convert to a simple digital format
> - Like converting to MIDI - you get notes, but lose the nuance
> - Each instrument becomes "loud" or "quiet" with few levels in between
>
> **FP4 approach:** Smart compression that keeps the dynamics
> - Still 4 bits, but they're used more intelligently
> - Quiet parts stay detailed, loud parts don't clip
> - The "floating point" part means the 4 bits adapt to the signal!
>
> **In AI terms:** FP4 uses 4 bits like INT4, but the floating-point representation better captures the weight distribution in neural networks. With Blackwell's specialized hardware, this runs at full speed!

---

## Part 1: Understanding FP4 Formats

### NVFP4 vs MXFP4

NVIDIA provides two FP4 formats:

| Format | Description | Best For |
|--------|-------------|----------|
| **NVFP4** | NVIDIA's proprietary format with dual-level scaling | Maximum performance |
| **MXFP4** | Open Compute Project standard (E2M1 with scaling) | Cross-platform compatibility |

### FP4 Bit Layout

```
FP4 (E2M1): [S][E E][M]
- S: Sign bit (1 bit)
- E: Exponent (2 bits) → 4 possible exponents
- M: Mantissa (1 bit) → 2 possible mantissa values

This gives us 8 positive and 8 negative values!
```

### Dual-Level Scaling (NVFP4's Secret)

The key innovation is **dual-level scaling**:
1. **Block-level scale**: Shared across a block of weights
2. **Sub-block scale**: Finer granularity within each block

This allows FP4 to adapt to different weight magnitudes across the model!

In [ ]:
import torch
import numpy as np
import matplotlib.pyplot as plt

print("="*60)
print("DGX Spark / Blackwell Environment Check")
print("="*60)

print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")

if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")
    
    # Check compute capability (Blackwell is 10.0+)
    cc = torch.cuda.get_device_capability()
    print(f"Compute Capability: {cc[0]}.{cc[1]}")
    
    if cc[0] >= 10:
        print("\n⭐ Blackwell GPU detected! FP4 tensor cores available!")
    else:
        print("\n⚠️  Non-Blackwell GPU detected. FP4 may run in emulation mode.")

In [ ]:
# Visualize FP4 representable values

def get_fp4_values():
    """
    Calculate all values representable by FP4 (E2M1) format.
    
    FP4 E2M1 format:
    - 1 sign bit
    - 2 exponent bits (bias = 1)
    - 1 mantissa bit
    """
    values = []
    bias = 1
    
    for sign in [1, -1]:
        for exp in range(4):  # 2 bits = 4 values
            for mant in range(2):  # 1 bit = 2 values
                if exp == 0:  # Subnormal
                    value = sign * (mant / 2) * (2 ** (1 - bias))
                else:  # Normal
                    value = sign * (1 + mant / 2) * (2 ** (exp - bias))
                values.append(value)
    
    return sorted(set(values))

def get_int4_values():
    """INT4 symmetric values (-7 to 7)."""
    return list(range(-7, 8))

fp4_values = get_fp4_values()
int4_values = get_int4_values()

print("FP4 (E2M1) Representable Values:")
print(f"  {fp4_values}")
print(f"  Count: {len(fp4_values)}")
print(f"  Range: [{min(fp4_values)}, {max(fp4_values)}]")

print("\nINT4 Symmetric Values:")
print(f"  {int4_values}")
print(f"  Count: {len(int4_values)}")
print(f"  Range: [{min(int4_values)}, {max(int4_values)}]")

In [ ]:
# Visualize the difference between INT4 and FP4

fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Plot representable values
ax = axes[0, 0]
ax.scatter(int4_values, [0]*len(int4_values), label='INT4', s=100, alpha=0.7)
ax.scatter(fp4_values, [0.1]*len(fp4_values), label='FP4 (E2M1)', s=100, alpha=0.7, marker='^')
ax.set_xlabel('Value')
ax.set_yticks([])
ax.set_title('Representable Values: INT4 vs FP4')
ax.legend()
ax.grid(True, alpha=0.3)

# Show density of representable values
ax = axes[0, 1]
bins = np.linspace(-8, 8, 50)
ax.hist(int4_values, bins=bins, alpha=0.5, label='INT4', density=True)
ax.hist(fp4_values, bins=bins, alpha=0.5, label='FP4', density=True)
ax.set_xlabel('Value')
ax.set_ylabel('Density')
ax.set_title('Distribution of Representable Values')
ax.legend()

# Quantization error comparison on Gaussian weights
ax = axes[1, 0]
torch.manual_seed(42)
weights = torch.randn(10000) * 0.5  # Typical weight distribution

def quantize_to_nearest(values, grid):
    """Quantize values to nearest grid point."""
    grid = np.array(grid)
    result = np.zeros_like(values)
    for i, v in enumerate(values):
        idx = np.argmin(np.abs(grid - v))
        result[i] = grid[idx]
    return result

# Scale weights to fit quantization range
weights_np = weights.numpy()
scale_int4 = max(abs(weights_np.max()), abs(weights_np.min())) / 7
scale_fp4 = max(abs(weights_np.max()), abs(weights_np.min())) / max(abs(max(fp4_values)), abs(min(fp4_values)))

# Quantize
int4_quant = quantize_to_nearest(weights_np / scale_int4, int4_values) * scale_int4
fp4_quant = quantize_to_nearest(weights_np / scale_fp4, fp4_values) * scale_fp4

# Compute errors
int4_error = np.abs(weights_np - int4_quant)
fp4_error = np.abs(weights_np - fp4_quant)

ax.hist(int4_error, bins=50, alpha=0.5, label=f'INT4 (mean={int4_error.mean():.4f})')
ax.hist(fp4_error, bins=50, alpha=0.5, label=f'FP4 (mean={fp4_error.mean():.4f})')
ax.set_xlabel('Absolute Error')
ax.set_ylabel('Count')
ax.set_title('Quantization Error Distribution')
ax.legend()

# Error vs weight magnitude
ax = axes[1, 1]
ax.scatter(np.abs(weights_np), int4_error, alpha=0.1, s=1, label='INT4')
ax.scatter(np.abs(weights_np), fp4_error, alpha=0.1, s=1, label='FP4')
ax.set_xlabel('Weight Magnitude')
ax.set_ylabel('Absolute Error')
ax.set_title('Error vs Weight Magnitude')
ax.legend()

plt.tight_layout()
plt.savefig('fp4_vs_int4.png', dpi=150, bbox_inches='tight')
plt.show()
plt.close(fig)  # Free memory from figure

print(f"\n📊 Quantization Error Summary:")
print(f"   INT4 Mean Error: {int4_error.mean():.6f}")
print(f"   FP4 Mean Error:  {fp4_error.mean():.6f}")
print(f"   FP4 has {(1 - fp4_error.mean()/int4_error.mean())*100:.1f}% lower error!")

### 🔍 Key Insight

FP4 has **non-uniform spacing** between values:
- More precision near zero (where most weights are)
- Coarser precision for larger magnitudes

This matches the typical weight distribution in neural networks!

---

## Part 2: Setting Up TensorRT Model Optimizer

NVIDIA's TensorRT Model Optimizer (ModelOpt) is the official tool for FP4 quantization.

In [ ]:
# Install TensorRT Model Optimizer
# Note: This should be pre-installed in the NGC container

try:
    import modelopt.torch.quantization as mtq
    from modelopt.torch.quantization import algorithms as quant_algo
    print("✅ TensorRT Model Optimizer is available!")
except ImportError:
    print("⚠️ TensorRT Model Optimizer not found.")
    print("   On DGX Spark, ensure you're using the correct NGC container:")
    print("   nvcr.io/nvidia/pytorch:25.03-py3 or newer")
    print("")
    print("Attempting installation...")
    
    try:
        import subprocess
        result = subprocess.run(
            ["pip", "install", "nvidia-modelopt[torch]", "--quiet"],
            capture_output=True,
            text=True
        )
        if result.returncode == 0:
            import modelopt.torch.quantization as mtq
            from modelopt.torch.quantization import algorithms as quant_algo
            print("✅ Installation successful!")
        else:
            raise ImportError(result.stderr)
    except Exception as e:
        print(f"❌ Installation failed: {e}")
        print("\nThis notebook requires ModelOpt for FP4 quantization.")
        print("Please use an NGC container with ModelOpt pre-installed, or install manually:")
        print("  pip install nvidia-modelopt[torch]")
        print("\nThe notebook will continue but some features may not work.")
        mtq = None
        quant_algo = None

In [ ]:
# Check ModelOpt version and available quantization configs

# Guard against mtq being None (from failed import in previous cell)
if mtq is None:
    print("⚠️  ModelOpt not available. Skipping config check.")
    print("   FP4/FP8 quantization features will not work in this session.")
    print("\n💡 Solutions:")
    print("   1. Use NGC container: nvcr.io/nvidia/pytorch:25.03-py3 or newer")
    print("   2. Install manually: pip install nvidia-modelopt[torch]")
else:
    import modelopt
    print(f"ModelOpt version: {modelopt.__version__}")
    print("\nAvailable quantization algorithms:")

    available_configs = [
        'INT8_SMOOTHQUANT_CFG',
        'INT4_AWQ_CFG',
        'FP8_DEFAULT_CFG',
        'NVFP4_DEFAULT_CFG',
        'MXFP4_DEFAULT_CFG',
    ]

    for config_name in available_configs:
        try:
            config = getattr(mtq, config_name, None)
            if config is not None:
                print(f"  ✅ {config_name}")
            else:
                print(f"  ❌ {config_name} (not available in this version)")
        except Exception as e:
            print(f"  ❌ {config_name} (error: {e})")

---

## Part 3: FP4 Quantization in Practice

Let's quantize a model using NVFP4!

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import gc
import time
import subprocess

# Use a model suitable for demonstration
# For production, use larger models like Llama-2-7B
model_id = "facebook/opt-350m"

print(f"Loading model: {model_id}")

# Clear buffer cache before loading models (DGX Spark best practice)
# This ensures maximum available unified memory
try:
    subprocess.run(
        "sudo sh -c 'sync; echo 3 > /proc/sys/vm/drop_caches'",
        shell=True, check=True, capture_output=True
    )
    print("Buffer cache cleared for optimal memory availability")
except subprocess.CalledProcessError:
    print("Note: Could not clear buffer cache (may need sudo)")

# Clear GPU memory
gc.collect()
torch.cuda.empty_cache()

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_id)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

# Load model in FP16
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.float16,
    device_map="cuda"
)

# Memory baseline
fp16_memory = torch.cuda.memory_allocated() / 1e9
print(f"FP16 model memory: {fp16_memory:.2f} GB")

In [ ]:
# Prepare calibration data
def get_calibration_dataloader(tokenizer, num_samples=128, seq_len=512):
    """
    Create calibration dataloader for FP4 quantization.
    
    Good calibration data is crucial for FP4 quality!
    """
    calibration_texts = [
        "The field of artificial intelligence has made remarkable progress in recent years.",
        "Large language models can understand and generate human-like text.",
        "Machine learning algorithms learn patterns from data.",
        "Neural networks are inspired by biological brain structure.",
        "Deep learning has revolutionized computer vision and NLP.",
        "The transformer architecture uses self-attention mechanisms.",
        "Quantization reduces model precision for efficient deployment.",
        "GPU acceleration enables fast neural network training.",
        "Transfer learning leverages pre-trained model knowledge.",
        "Attention mechanisms help models focus on relevant information.",
        "The history of computing spans several decades of innovation.",
        "Scientific research requires careful methodology and analysis.",
        "Climate change affects ecosystems around the world.",
        "Medical advances have improved human health outcomes.",
        "Space exploration continues to push boundaries.",
        "Economic factors influence market behavior.",
    ]
    
    # Extend to desired number of samples
    extended = (calibration_texts * ((num_samples // len(calibration_texts)) + 1))[:num_samples]
    
    # Tokenize
    encodings = tokenizer(
        extended,
        truncation=True,
        max_length=seq_len,
        padding='max_length',
        return_tensors='pt'
    )
    
    # Create simple dataloader
    from torch.utils.data import DataLoader, TensorDataset
    dataset = TensorDataset(encodings.input_ids, encodings.attention_mask)
    dataloader = DataLoader(dataset, batch_size=8)
    
    return dataloader

calib_dataloader = get_calibration_dataloader(tokenizer)
print(f"Calibration dataloader ready: {len(calib_dataloader)} batches")

In [ ]:
# Define calibration forward function
def calibration_forward(model):
    """
    Run calibration forward passes.
    
    This function is called by ModelOpt during quantization
    to collect activation statistics.
    """
    model.eval()
    with torch.no_grad():
        for batch_idx, (input_ids, attention_mask) in enumerate(calib_dataloader):
            input_ids = input_ids.to(model.device)
            attention_mask = attention_mask.to(model.device)
            
            _ = model(
                input_ids=input_ids,
                attention_mask=attention_mask
            )
            
            if batch_idx >= 15:  # Use ~128 samples
                break

print("Calibration function defined!")

In [ ]:
# Apply NVFP4 quantization
print("="*60)
print("Applying NVFP4 Quantization (Blackwell Exclusive!)")
print("="*60)

start_time = time.time()

try:
    # Get NVFP4 configuration
    nvfp4_config = mtq.NVFP4_DEFAULT_CFG
    
    # Apply quantization
    model_fp4 = mtq.quantize(
        model,
        nvfp4_config,
        forward_loop=calibration_forward
    )
    
    quant_time = time.time() - start_time
    print(f"\n✓ Quantization complete in {quant_time:.1f}s")
    
    # Check memory
    gc.collect()
    torch.cuda.empty_cache()
    fp4_memory = torch.cuda.memory_allocated() / 1e9
    print(f"FP4 model memory: {fp4_memory:.2f} GB")
    print(f"Memory reduction: {fp16_memory/fp4_memory:.2f}x")
    
except Exception as e:
    print(f"\n⚠️  NVFP4 quantization not available: {e}")
    print("\nThis typically means:")
    print("  1. You're not on a Blackwell GPU, or")
    print("  2. ModelOpt needs to be updated")
    print("\nWe'll demonstrate with FP8 instead...")
    
    # Fallback to FP8 for demonstration
    try:
        fp8_config = mtq.FP8_DEFAULT_CFG
        model_fp4 = mtq.quantize(
            model,
            fp8_config,
            forward_loop=calibration_forward
        )
        print("\n✓ FP8 quantization complete (as fallback)")
    except Exception as e2:
        print(f"FP8 also failed: {e2}")
        model_fp4 = model

In [ ]:
# Test inference with quantized model
print("\nTesting inference with quantized model...")

prompt = "The future of artificial intelligence"
inputs = tokenizer(prompt, return_tensors="pt").to(model_fp4.device)

model_fp4.eval()
with torch.no_grad():
    # Warmup
    _ = model_fp4.generate(**inputs, max_new_tokens=10, do_sample=False)
    
    # Benchmark
    torch.cuda.synchronize()
    start = time.perf_counter()
    
    outputs = model_fp4.generate(
        **inputs,
        max_new_tokens=50,
        do_sample=False,
        pad_token_id=tokenizer.pad_token_id
    )
    
    torch.cuda.synchronize()
    inference_time = time.perf_counter() - start

generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
tokens_generated = outputs.shape[1] - inputs['input_ids'].shape[1]

print(f"\nPrompt: {prompt}")
print(f"Generated: {generated_text}")
print(f"\nTokens generated: {tokens_generated}")
print(f"Time: {inference_time:.2f}s")
print(f"Speed: {tokens_generated/inference_time:.1f} tok/s")

---

## Part 4: MXFP4 Quantization

MXFP4 (Microscaling FP4) is an open standard from the Open Compute Project.

In [ ]:
# Clean up previous model
del model_fp4
gc.collect()
torch.cuda.empty_cache()

# Reload base model
print("Reloading base model for MXFP4 quantization...")

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.float16,
    device_map="cuda"
)

In [ ]:
# Apply MXFP4 quantization
print("="*60)
print("Applying MXFP4 Quantization (Open Compute Standard)")
print("="*60)

start_time = time.time()

try:
    mxfp4_config = mtq.MXFP4_DEFAULT_CFG
    
    model_mxfp4 = mtq.quantize(
        model,
        mxfp4_config,
        forward_loop=calibration_forward
    )
    
    quant_time = time.time() - start_time
    print(f"\n✓ MXFP4 quantization complete in {quant_time:.1f}s")
    
except Exception as e:
    print(f"\n⚠️  MXFP4 quantization not available: {e}")
    model_mxfp4 = model

---

## Part 5: Quality Evaluation

Let's measure the quality impact of FP4 quantization.

In [ ]:
import math
from tqdm import tqdm

def calculate_perplexity(model, tokenizer, texts, max_length=256):
    """Calculate perplexity on texts."""
    model.eval()
    total_loss = 0
    total_tokens = 0
    
    with torch.no_grad():
        for text in tqdm(texts, desc="Evaluating", leave=False):
            encodings = tokenizer(
                text, 
                return_tensors='pt', 
                truncation=True, 
                max_length=max_length
            )
            input_ids = encodings.input_ids.to(model.device)
            
            if input_ids.size(1) < 2:
                continue
            
            outputs = model(input_ids, labels=input_ids)
            loss = outputs.loss.item()
            num_tokens = input_ids.size(1) - 1
            
            total_loss += loss * num_tokens
            total_tokens += num_tokens
    
    return math.exp(total_loss / total_tokens)

# Evaluation texts
eval_texts = [
    "The quick brown fox jumps over the lazy dog in the garden.",
    "Machine learning enables computers to learn from experience.",
    "Scientists discovered a new particle at the hadron collider.",
    "The ancient civilization built impressive structures.",
    "Modern medicine has extended human lifespan significantly.",
    "Climate models predict significant changes this century.",
    "The economy showed resilience despite global challenges.",
    "Space agencies plan missions to explore distant planets.",
    "Renewable energy adoption continues to accelerate.",
    "Digital transformation reshapes business operations.",
]

In [ ]:
# Compare FP16 vs FP4 perplexity
print("Evaluating model quality...")
print("="*60)

# Load fresh FP16 model for baseline
print("\nLoading FP16 baseline...")
model_fp16 = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.float16,
    device_map="cuda"
)

print("Calculating FP16 perplexity...")
ppl_fp16 = calculate_perplexity(model_fp16, tokenizer, eval_texts)
print(f"FP16 Perplexity: {ppl_fp16:.2f}")

del model_fp16
gc.collect()
torch.cuda.empty_cache()

In [ ]:
# Evaluate FP4 model
print("\nCalculating FP4 perplexity...")

try:
    if 'model_mxfp4' in globals() and model_mxfp4 is not None:
        ppl_fp4 = calculate_perplexity(model_mxfp4, tokenizer, eval_texts)
        print(f"FP4 Perplexity: {ppl_fp4:.2f}")
        
        # Calculate degradation
        ppl_increase = ppl_fp4 - ppl_fp16
        ppl_percent = (ppl_increase / ppl_fp16) * 100
        
        print(f"\n📊 Quality Summary:")
        print(f"   FP16 Perplexity: {ppl_fp16:.2f}")
        print(f"   FP4 Perplexity:  {ppl_fp4:.2f}")
        print(f"   Increase: +{ppl_increase:.2f} ({ppl_percent:.1f}%)")
        
        if ppl_percent < 1:
            print("\n🎉 Excellent! Less than 1% quality degradation!")
        elif ppl_percent < 5:
            print("\n✓ Good quality preservation (<5% degradation)")
        else:
            print("\n⚠️ Consider using FP8 for better quality")
            
except Exception as e:
    print(f"Could not evaluate FP4 model: {e}")
    ppl_fp4 = ppl_fp16 * 1.01  # Estimated for visualization

In [ ]:
# Visualize the results
fig, axes = plt.subplots(1, 3, figsize=(15, 5))

precisions = ['FP16', 'FP8', 'NVFP4', 'MXFP4']
# Estimated values based on typical results
memory_gb = [fp16_memory, fp16_memory/2, fp16_memory/3.5, fp16_memory/3.5]
perplexities = [ppl_fp16, ppl_fp16*1.002, ppl_fp16*1.005, ppl_fp16*1.008]
speedups = [1.0, 1.5, 2.5, 2.4]

colors = ['#2196F3', '#4CAF50', '#FF9800', '#F44336']

# Memory
axes[0].bar(precisions, memory_gb, color=colors)
axes[0].set_ylabel('Memory (GB)')
axes[0].set_title('Memory Usage')
for i, v in enumerate(memory_gb):
    axes[0].text(i, v + 0.01, f'{v:.2f}', ha='center')

# Perplexity
axes[1].bar(precisions, perplexities, color=colors)
axes[1].set_ylabel('Perplexity')
axes[1].set_title('Quality (Lower is Better)')
for i, v in enumerate(perplexities):
    axes[1].text(i, v + 0.5, f'{v:.1f}', ha='center')
axes[1].set_ylim(min(perplexities)*0.95, max(perplexities)*1.05)

# Speedup
axes[2].bar(precisions, speedups, color=colors)
axes[2].set_ylabel('Relative Speedup')
axes[2].set_title('Inference Speed')
for i, v in enumerate(speedups):
    axes[2].text(i, v + 0.05, f'{v:.1f}x', ha='center')

plt.tight_layout()
plt.savefig('fp4_comparison.png', dpi=150, bbox_inches='tight')
plt.show()
plt.close(fig)  # Free memory from figure

---

## Part 6: Expected Performance on DGX Spark

Based on NVIDIA's benchmarks, here's what you can expect with FP4 on DGX Spark:

| Model | Precision | Memory | Prefill (tok/s) | Decode (tok/s) |
|-------|-----------|--------|-----------------|----------------|
| Llama 3.1 8B | FP16 | 16 GB | ~3,000 | ~20 |
| Llama 3.1 8B | NVFP4 | 4.5 GB | ~10,000 | ~39 |
| Llama 3.1 70B | FP16 | 140 GB | N/A (too big) | N/A |
| Llama 3.1 70B | NVFP4 | 40 GB | ~1,200 | ~12 |

### Key Takeaways:

1. **3.5× memory reduction** allows fitting larger models
2. **~3× prefill speedup** from native FP4 tensor cores
3. **2× decode speedup** from reduced memory bandwidth
4. **<1% accuracy loss** with proper calibration

---

## ✋ Try It Yourself

### Exercise 1: Quantize Llama 2 7B with NVFP4

Apply FP4 quantization to a larger model and measure the quality/speed tradeoffs.

<details>
<summary>💡 Hint</summary>

```python
model_id = "meta-llama/Llama-2-7b-hf"
# Follow the same quantization steps
# You'll need to log in to HuggingFace first
```
</details>

In [ ]:
# TODO: Quantize a larger model with FP4
# YOUR CODE HERE

### Exercise 2: Compare Calibration Data Quality

Try quantizing with different calibration datasets:
1. Random text
2. Domain-specific text
3. Code samples

How does calibration data affect FP4 quality?

<details>
<summary>💡 Hint</summary>

Create three different `get_calibration_dataloader` functions with different text sources.
</details>

In [ ]:
# TODO: Compare calibration data quality
# YOUR CODE HERE

---

## ⚠️ Common Mistakes

### Mistake 1: Insufficient Calibration Data

```python
# ❌ Wrong: Too few samples
calib_data = ["Hello world"]  # Only 1 sample!

# ✅ Right: Use diverse, representative data
calib_data = load_diverse_samples(128)  # 128+ samples
```

**Why:** FP4 needs good activation statistics for dual-level scaling.

### Mistake 2: Running on Non-Blackwell Hardware

```python
# ❌ Wrong: Expecting FP4 speed on older GPUs
# FP4 runs in software emulation, much slower!

# ✅ Right: Verify Blackwell hardware
cc = torch.cuda.get_device_capability()
assert cc[0] >= 10, "FP4 tensor cores require Blackwell!"
```

**Why:** FP4 tensor cores are exclusive to Blackwell architecture.

### Mistake 3: Not Clearing Memory Before Quantization

```python
# ❌ Wrong: Quantizing with other models in memory
model_fp4 = mtq.quantize(model, config)  # May OOM!

# ✅ Right: Clear memory first
gc.collect()
torch.cuda.empty_cache()
model_fp4 = mtq.quantize(model, config)
```

**Why:** Quantization temporarily needs extra memory for calibration.

---

## 🎉 Checkpoint

You've learned:

- ✅ **FP4 is your DGX Spark superpower**: Native tensor core support!
- ✅ **NVFP4 vs MXFP4**: NVIDIA's format vs Open Compute standard
- ✅ **3.5× compression**: With <1% accuracy loss
- ✅ **Dual-level scaling**: The secret to FP4 quality
- ✅ **TensorRT Model Optimizer**: The official tool for FP4

---

## 🚀 Challenge (Optional)

**Run Llama 70B on DGX Spark with FP4**

The ultimate test of FP4:
1. Load Llama 70B (normally 140GB!)
2. Quantize to FP4 (~40GB)
3. Run inference on your desktop DGX Spark!

This is impossible on any other desktop hardware!

```python
# Clear system cache first
!sudo sh -c 'sync; echo 3 > /proc/sys/vm/drop_caches'

model_id = "meta-llama/Llama-2-70b-hf"
# YOUR CODE HERE
```

---

## 📖 Further Reading

- [NVIDIA TensorRT Model Optimizer](https://developer.nvidia.com/tensorrt)
- [Blackwell Architecture Whitepaper](https://www.nvidia.com/en-us/data-center/technologies/blackwell-architecture/)
- [Open Compute Project MX Specification](https://www.opencompute.org/documents/ocp-microscaling-formats-mx-v1-0-spec-final-pdf)
- [FP4 LLM Paper](https://arxiv.org/abs/2310.16836)

---

## 🧹 Cleanup

In [ ]:
# Clean up models explicitly
import gc
import torch

# List of model variables to clean up
_models_to_delete = ['model', 'model_mxfp4', 'model_fp4', 'model_fp16']

for _var_name in _models_to_delete:
    if _var_name in dir():
        try:
            # Get the variable and delete it
            _var = eval(_var_name)
            if _var is not None:
                del _var
            # Also delete from local scope
            exec(f"del {_var_name}")
        except (NameError, TypeError):
            pass  # Variable doesn't exist or can't be deleted

# Force garbage collection
gc.collect()

# Clear CUDA cache
if torch.cuda.is_available():
    torch.cuda.empty_cache()
    torch.cuda.synchronize()

print("✅ Cleanup complete!")
if torch.cuda.is_available():
    print(f"GPU memory after cleanup: {torch.cuda.memory_allocated()/1e9:.2f} GB")

---

## Next Steps

In the final notebook, we'll create a **comprehensive quality benchmark suite** to compare all quantization methods!

➡️ Continue to: [06-quality-benchmark-suite.ipynb](06-quality-benchmark-suite.ipynb)